In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import xmltodict
import os
import json
import pickle
from lxml import etree


In [3]:
# mode = int(metadata['mode']) if metadata['mode'] is not None else 1
# beats_in_measure = int(metadata['beats_in_measure'])

# melody, chord = segments_parser(segments, mode, beats_in_measure)

In [4]:
from pathlib import Path

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version
orig_path = version_path/'midi_sources'

In [6]:
from fastai.data_block import get_files

In [7]:
h_path = orig_path/'hooktheory'

In [8]:
files = get_files(h_path, extensions=['.xml'], recurse=True); files[:10]

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/what-a-day/kiefer/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/pre-chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/freedom/chorus.xml')]

In [9]:
# Loading from specific file
# keywords = ['get-lucky', 'daft-punk', 'pre-chorus']
# keywords = ['skrillex', 'scary']
keywords = ['idina', 'verse', 'let']
# keywords = ['game-of-thrones', 'intro', 'ramin']
# keywords = ['kiss-from-a-rose', 'seal']
def contains_keywords(f): return all([k in str(f) for k in keywords])
search = [f for f in files if contains_keywords(f)]; search

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/intro-and-verse.xml')]

In [10]:
from src.tab_parser import *

In [11]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [12]:
def parse_file(file_path):
    content = load_data(file_path)
    root = xml_parser(content)
    metadata, version = get_metadata(root)
    segments, num_measures = get_lead_sheet(root, version)
    
    song = HSong.parse(metadata, segments)
    return song

In [13]:
import music21

### Create config file

In [14]:
config_opts = dict(sustain=True, sep_octave=True, note_octave=4, chord_octave=3, 
              ts='4/4', ks=0, bpm=120, freq=2, pad_idx=-1, none_idx=0, sus_idx=2, hit_idx=1)

class Config(object):
    def __init__(self, d): self.__dict__ = d
        
config = Config(config_opts)

### Constants

In [15]:
MODE_TO_KEYOFFSET = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11
#     '5': -5,
#     '6': -3,
#     '7': -1
}

In [16]:
PITCH_TO_SD = {
    0: '1',
    1: '1#',
    2: '2',
    3: '2#',
    4: '3',
    5: '4',
    6: '4#',
    7: '5',
    8: '5#',
    9: '6',
    10:'6#',
    11:'7',
}

SD_TO_PITCH = {v:k for k,v in PITCH_TO_SD.items()}

### Classes

In [17]:
from dataclasses import dataclass
import dataclasses
from typing import Dict, Any, AnyStr, List, Sequence, TypeVar, Tuple, Optional, Union

In [18]:
def parse(cls, d):
    cls_keys = cls.__dataclass_fields__.keys()
    kwargs = {key:d[key] for key in cls_keys}
    return cls(**kwargs)

@dataclass
class Base:
    @classmethod
    def from_dict(cls, d):
        cls_keys = cls.__dataclass_fields__.keys()
        kwargs = {key:d[key] for key in cls_keys}
        return cls(**kwargs)
    
    @classmethod
    def parse(cls, d):
        return cls.from_dict(d)

In [19]:
@dataclass
class HMetadata(Base):
    title:str
    BPM:str='120'
    beats_in_measure:str='4'
    key:str='C'
    mode:str='1'

In [20]:
@dataclass
class HNote(Base):
    beat_abs:float
    measure:float
    beat:float
    duration:float
    scale:str
    octave: str
        
    def to_m21(self)->music21.note.Note:
#         if self.scale_degree == 'rest': return None, None
#             n = music21.note.Rest(quarterLength=note_length)
        pitch = self.pitch() + 12*(int(self.octave)+config.note_octave)
        n = music21.note.Note(pitch, quarterLength=self.duration)
#         key.KeySignature(-1)
        return n, float(self.beat_abs)
    
    def pitch(self):
        return SD_TO_PITCH[self.scale]
    
    def end_time(self):
        return self.duration + self.beat_abs
    
    @classmethod
    def parse(cls, d, mode, key_offset):
#         if key_offset > 5: key_offset = key_offset-12
        parsed = roman_to_symbol.hnote_parser(d, mode, key_offset)
        pitch = parsed['pitch']
        scale_degree = PITCH_TO_SD[int((pitch) % 12)]
        octave = (pitch // 12) + 1
        m = {
            'scale': scale_degree,
            'octave': octave,
            'duration': float(d['note_length']),
            'measure': float(d['start_measure']),
            'beat': float(d['start_beat']),
            'beat_abs': float(d['start_beat_abs']),
        }
        return cls.from_dict({**d, **m})

In [21]:
def last_comp(comp):
    if comp is None: return config.none_idx
    return int(comp[-1])



@dataclass
class HChord(Base):
    # ht relative
    scale:int
    base:int
    sus:int
    
    # ht tempo
    duration:float # (AS) TODO: convert to float
    measure:float
    beat:float
    beat_abs:float # (AS) TODO: convert to float
        
    # abs
    composition:List[int]
    symbol:str=None
    quality:str=None
        
    def end_time(self):
        return self.duration + self.beat_abs
        
    def to_m21(self)->music21.chord.Chord:
        notes = [n+config.chord_octave*12 for n in self.composition]
        c = music21.chord.Chord(notes, quarterLength=self.duration)
        return c, float(self.beat_abs)

    @classmethod
    def parse(cls, d, mode, key_offset, reset_to_base=True, remove_emb=True):
        if remove_emb:
            d = d.copy()
            d['sus'] = None
            d['emb'] = None
            if isinstance(d.get('fb'), str) and int(d['fb'][0]) > 7: d['fb'] = '9'
        parsed = roman_to_symbol.hchord_parser(d, mode, 0)
        
        # After offset, let's reset the chord to be the lowest possible offset on new scale
        if reset_to_base:
            lowest = min(parsed['composition'])
            if lowest >= 12:
                parsed = roman_to_symbol.chord_key_shifting(parsed, -12)
                # reset chord name
                new_s = roman_to_symbol.chord_to_string(parsed)
                parsed['symbol'] = new_s
        
        parsed['composition'] = parsed['composition'].astype(int).tolist()
        
        m = {
            'scale': (int(d['sd']) - (1-int(mode))) % 8,
            'base': last_comp(d['fb']),
            'sus': last_comp(d['sus']),
            'duration': float(d['chord_duration']),
            'measure': float(d['start_measure']),
            'beat': float(d['start_beat']),
            'beat_abs': float(d['start_beat_abs']),
        }
        
        return cls.from_dict({**parsed, **m})

In [22]:
def default_stream(cls=music21.stream.Score, ts='4/4', bpm=120, ks=0):
    # (AS) TODO: use config ts or metadata
    s = cls()
    s.append(music21.instrument.Piano())
    s.append(music21.meter.TimeSignature(ts))
    s.append(music21.tempo.MetronomeMark(number=bpm))
#     s.append(music21.key.KeySignature(ks))
    s.append(music21.key.Key('C'))
    return s

In [23]:
@dataclass
class HPart(Base):
    notes: List[HNote]
    chords: List[HChord]
        
    @classmethod
    def parse(cls, d, metadata):
        mode = metadata['mode'] or '1'
        key_offset = MODE_TO_KEYOFFSET.get(mode, 0)
        ns = [HNote.parse(n, mode, key_offset) for n in d.get('notes', []) if n['scale_degree'] != 'rest']
        cs = [HChord.parse(c, mode, key_offset) for c in d.get('chords', []) if c['sd'] != 'rest']
        return cls(notes=ns, chords=cs)
    
    
    def duration(self):
        c_last = self.chords[-1].end_time()
        n_last = self.notes[-1].end_time()
        return max(c_last, n_last)
    
    def to_m21(self)->music21.stream.Stream:
        mc = music21.stream.Part()
        mn = music21.stream.Part()
        
        cm21 = [c.to_m21() for c in self.chords]
        for c,d in cm21: mc.insert(d,c)
            
        nm21 = [n.to_m21() for n in self.notes]
        for n,d in nm21: mn.insert(d,n)
        return mn, mc
        
    def min_pitch(self):
        return min([n.pitch for n in self.notes])

In [24]:
@dataclass
class HSong(Base):
    metadata: HMetadata
    parts: List[HPart]
    
    @classmethod
    def parse(cls, metadata, segments):
        m = HMetadata.parse(metadata)
        ps = [HPart.parse(s, metadata) for s in segments]
        return cls(metadata=m, parts=ps)
    
    def duration(self):
        return sum(p.duration() for p in self.parts)
    
    def to_stream(self):
        s = default_stream()
        pc = music21.stream.Part()
        pn = music21.stream.Part()
        
        for p in self.parts:
            mn, mc = p.to_m21()
            pn.append(mn)
            pc.append(mc)
            
        s.insert(0, pn)
        s.insert(0, pc)

#         s.flat.makeNotation(inPlace=True)
        s = s.transpose(0) # hack to get accidentals right. Above method does not work
        # music21 stream
        return s

In [25]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [26]:
import numpy as np
from collections import Counter, defaultdict

In [27]:
from src.roman_to_symbol import *

In [132]:
all_symbols = []
all_notes = []
all_map = []
jumps = defaultdict(list)

In [133]:
# file_path = Path('data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml')
# content = load_data(file_path)
# root = xml_parser(content)
# metadata, version = get_metadata(root)
# segments, num_measures = get_lead_sheet(root, version)
# cdata['sus'] = None
# cdata['emb'] = 'add9'
# cdata = segments[-1]['chords'][-2]; cdata

In [134]:
for f in files:
    try: s = parse_file(f)
    except Exception as e: 
        print('Exception:', e)
        continue
    for p in s.parts:
        for c in p.chords:
            if len(c.composition) > 4: continue
            all_symbols.append(c.symbol)
            chords = list(([i%12 for i in sorted(c.composition)]))
            all_notes.append(str(chords))
            all_map.append(c.symbol + ' - ' + str(chords))
            j = np.diff(sorted(c.composition))
            jumps[str(j)].append((c.composition, c.symbol))

Exception: 'NoneType' object has no attribute 'text'
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.
Exception: Unico

In [135]:
jumps_count = {k:len(v) for k,v in jumps.items()}; len(jumps_count)

25

In [136]:
jumps['[4 1 4]']

[([5, 9, 0, 4], 'Fmaj7 | C'),
 ([5, 9, 0, 4], 'Fmaj7 | C'),
 ([5, 9, 0, 4], 'Fmaj7 | C'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([8, 12, 3, 7], 'Abmaj7 | Eb'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([13, 17, 8, 12], 'Dbmaj7 | Ab'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([10, 14, 5, 9], 'Bbmaj7 | F'),
 ([10, 14, 5, 9], 'Bbmaj7 | F'),
 ([6, 10, 1, 5], 'Gbmaj7 | Db'),
 ([15, 19, 10, 14], 'Ebmaj7 | Bb'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([8, 12, 3, 7], 'Abmaj7 | Eb'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([15, 19, 10, 14], 'Ebmaj7 | Bb'),
 ([15, 19, 10, 14], 'Ebmaj7 | Bb'),
 ([15, 19, 10, 14], 'Ebmaj7 | Bb'),
 ([15, 19, 10, 14], 'Ebmaj7 | Bb'),
 ([10, 14, 5, 9], 'Bbmaj7 | F'),
 ([10, 14, 5, 9], 'Bbmaj7 | F'),
 ([5, 9, 0, 4], 'Fmaj7 | C'),
 ([5, 9, 0, 4], 'Fmaj7 | C'),
 ([5, 9, 0, 4], 'Fmaj7 | C'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([12, 16, 7, 11], 'Cmaj7 | G'),
 ([9, 13, 4, 8], 'Amaj7 | E'),
 ([5, 9, 0, 4

In [143]:
sum(jumps_count.values())

316795

In [144]:
{k:v/316795 for k,v in jumps_count.items()}

{'[3 4]': 0.24016161871241654,
 '[3 2 3]': 0.0016951025110876119,
 '[2 3 4]': 0.005814485708423429,
 '[3 4 2]': 0.0020707397528370083,
 '[3 4 3]': 0.061326725484935056,
 '[4 3 4]': 0.038220300194131855,
 '[4 3]': 0.48079672974636595,
 '[4 3 3]': 0.03310973973705393,
 '[3 5]': 0.03778153064284474,
 '[5 4]': 0.02551176628419009,
 '[3 3]': 0.010527312615413753,
 '[3 3 4]': 0.0037216496472482203,
 '[3 3 2]': 0.0029735317792263134,
 '[3 2 4]': 0.002130715446897836,
 '[4 3 2]': 0.008728041793588914,
 '[3 4 1]': 0.0013510314241070723,
 '[6 3]': 0.002001294212345523,
 '[1 4 3]': 0.002373774838618034,
 '[2 4 3]': 0.004883284142742152,
 '[4 5]': 0.016660616487002636,
 '[5 3]': 0.013021038842153443,
 '[3 6]': 0.0019413185182846951,
 '[4 2 3]': 0.0010132735680803042,
 '[2 3 3]': 0.0011426948026326174,
 '[4 1 4]': 0.0010416831073722754}

In [137]:
jumps_count

{'[3 4]': 76082,
 '[3 2 3]': 537,
 '[2 3 4]': 1842,
 '[3 4 2]': 656,
 '[3 4 3]': 19428,
 '[4 3 4]': 12108,
 '[4 3]': 152314,
 '[4 3 3]': 10489,
 '[3 5]': 11969,
 '[5 4]': 8082,
 '[3 3]': 3335,
 '[3 3 4]': 1179,
 '[3 3 2]': 942,
 '[3 2 4]': 675,
 '[4 3 2]': 2765,
 '[3 4 1]': 428,
 '[6 3]': 634,
 '[1 4 3]': 752,
 '[2 4 3]': 1547,
 '[4 5]': 5278,
 '[5 3]': 4125,
 '[3 6]': 615,
 '[4 2 3]': 321,
 '[2 3 3]': 362,
 '[4 1 4]': 330}

In [138]:
# quality - 

In [139]:
symbol_count = Counter(all_symbols); len(symbol_count)

287

In [140]:
symbol_count.most_common()

[('C', 39422),
 ('cm', 33977),
 ('F', 28348),
 ('G', 27719),
 ('Bb', 20217),
 ('Ab', 17270),
 ('am', 14355),
 ('Eb', 11450),
 ('fm', 8082),
 ('dm', 6761),
 ('gm', 6209),
 ('em', 4856),
 ('G7', 4831),
 ('cm7', 4481),
 ('Abmaj7', 4199),
 ('dm7', 3683),
 ('C | E', 3314),
 ('G | B', 3179),
 ('am7', 3109),
 ('Fmaj7', 3040),
 ('fm7', 2727),
 ('gm7', 2605),
 ('D', 2503),
 ('Cmaj7', 2435),
 ('C | G', 2369),
 ('em7', 2066),
 ('Db', 1992),
 ('cm | Eb', 1738),
 ('F | C', 1718),
 ('cm | G', 1714),
 ('F | A', 1494),
 ('C7', 1352),
 ('Bb | D', 1269),
 ('E', 1238),
 ('G | D', 1216),
 ('Ebmaj7', 1008),
 ('Eb | G', 973),
 ('Eb | Bb', 966),
 ('gm | Bb', 958),
 ('A', 958),
 ('bo', 930),
 ('fm | Ab', 897),
 ('D7', 865),
 ('Ab | C', 819),
 ('Bb7', 750),
 ('Bb | F', 738),
 ('cm7 | Bb', 730),
 ('F7', 724),
 ('bbm', 657),
 ('Gb', 633),
 ('fm | C', 619),
 ('E7', 613),
 ('gm7 | Bb', 593),
 ('Ab | Eb', 581),
 ('B', 564),
 ('A7', 558),
 ('am | C', 537),
 ('gbo', 502),
 ('Dbmaj7', 501),
 ('dm | F', 496),
 ('do', 4

In [141]:
note_count = Counter(all_notes); len(note_count)

287

In [142]:
note_count.most_common()

[('[0, 4, 7]', 39422),
 ('[0, 3, 7]', 33977),
 ('[5, 9, 0]', 28348),
 ('[7, 11, 2]', 27719),
 ('[10, 2, 5]', 20217),
 ('[8, 0, 3]', 17270),
 ('[9, 0, 4]', 14355),
 ('[3, 7, 10]', 11450),
 ('[5, 8, 0]', 8082),
 ('[2, 5, 9]', 6761),
 ('[7, 10, 2]', 6209),
 ('[4, 7, 11]', 4856),
 ('[7, 11, 2, 5]', 4831),
 ('[0, 3, 7, 10]', 4481),
 ('[8, 0, 3, 7]', 4199),
 ('[2, 5, 9, 0]', 3683),
 ('[4, 7, 0]', 3314),
 ('[11, 2, 7]', 3179),
 ('[9, 0, 4, 7]', 3109),
 ('[5, 9, 0, 4]', 3040),
 ('[5, 8, 0, 3]', 2727),
 ('[7, 10, 2, 5]', 2605),
 ('[2, 6, 9]', 2503),
 ('[0, 4, 7, 11]', 2435),
 ('[7, 0, 4]', 2369),
 ('[4, 7, 11, 2]', 2066),
 ('[1, 5, 8]', 1992),
 ('[3, 7, 0]', 1738),
 ('[0, 5, 9]', 1718),
 ('[7, 0, 3]', 1714),
 ('[9, 0, 5]', 1494),
 ('[0, 4, 7, 10]', 1352),
 ('[2, 5, 10]', 1269),
 ('[4, 8, 11]', 1238),
 ('[2, 7, 11]', 1216),
 ('[3, 7, 10, 2]', 1008),
 ('[7, 10, 3]', 973),
 ('[10, 3, 7]', 966),
 ('[10, 2, 7]', 958),
 ('[9, 1, 4]', 958),
 ('[11, 2, 5]', 930),
 ('[8, 0, 5]', 897),
 ('[2, 6, 9, 0]', 

chord without inversion

In [ ]:


# def hchord_parser(chord, mode, key_offset):
#     if chord['sd'] == 'rest': return None

#     # extract basic info
#     sd = int(chord['sd']) - 1     # root
#     fb = chord['fb']              # tension & inversion
#     sec = chord['sec']            # secondary chord
#     borrowed = chord.get('borrowed', None)  # borrowed mode

#     # determine the mode
#     borrowed = is_int(borrowed)
#     chord_key = MODE_TO_KEY[int(mode)] if borrowed is None else borrowed
#     chord_key = 6 if chord_key > 6 else chord_key
#     chord_key = -6 if chord_key < -6 else chord_key

#     # secondary chord
#     sec_offset = 0
#     if sec:
#         # switch to 'sec' degree note within the current mode
#         scale = KEY_TO_SCALE[MODE_TO_KEY[int(mode)]]
#         new_key_note = scale[int(sec) - 1]

#         # set that note to new key
#         new_key = VAL_TO_NAME[new_key_note][0]

#         # get the key shift offset
#         sec_offset = NOTE_TO_OFFSET[new_key]
#         chord_key = 0

#     # determine the scale according to the key(mode) of the chord
#     scale = get_scale(chord_key)

#     # set compositional notes
#     comp, chord_type = set_compositions(scale, fb, sd)

#     # determine the quality by triads or seventh
#     # (9, 11, 13-th chords are seen as seventh)
#     comp_t = comp[0:3] if chord_type is 5 else comp[0:4]
#     quality = get_quality(comp_t)

#     # add shift from secondary chords
#     comp = (comp + sec_offset)

#     # set compvec (for sus/add/omit)
#     comp_vec = comp_to_compvec(comp)

#     # sus (omit 3)
#     sus = chord.get('sus', None)
#     comp_vec = set_sus(comp_vec, scale, sd, sus)

#     # emb (add/omit)
#     if 'emb' in chord:
#         emb = chord['emb']
#         comp_vec, alter_info, emb_info = set_emb(comp_vec, scale, sd, emb)
#     else:
#         emb_info = []
#         alter_info = []

#     # alter (won't change the quality)
#     alter_info = alter_info if len(alter_info) else chord.get('alternate']
#     comp_vec, alter_map = set_alter(comp_vec, alter_info)

#     # set inversion (won't change the root, but bass)
#     inv = get_num_inversion(fb)
#     comp_vec = set_inversion(comp_vec, inv)

#     # set result
#     comp = compvec_to_comp(comp_vec)
#     root = (comp[0] + 120) % 12   # for chord name
#     bass = np.nanmin(comp)         # for bass (real root)

#     data = OrderedDict([
#         # basic compositions
#         ('root', root),
#         ('bass', bass),
#         ('comp_vec', comp_vec),
#         ('composition', comp),

#         # basic info
#         ('quality', quality),
#         ('chord_type', chord_type),
#         ('chord_mode', chord_key),
        
#         # additional info
#         ('inv', inv),
#         ('sus', sus),
#         ('alter', alter_info),
#         ('emb', emb_info),
#         ('alter_map', alter_map),
#         ])

#     # key shifting of the symbol
#     data = chord_key_shifting(data, key_offset)

#     # set chord name
#     data['symbol'] = chord_to_string(data)
#     return data

In [180]:
VAL_TO_NAME = {
    1: 
}

In [ ]:
def chord_to_string(data):
    base = to_name(data['root']) + data['quality']
    if data['quality'] in ['m', 'ø', 'o']:
        base = base.lower()
    base += ('' if data['chord_type'] is 5 else str(data['chord_type']))
    if data['inv']:
        base += (' | ' + to_name(data['bass']))
    if data['sus'] is not None:
        base += (' ' + data['sus'])
    if data['emb'] is not None:
        for e in data['emb']:
            base += ' (%s)' % e
    if data['alter'] is not None:
        for a in data['alter']:
            base += ' (%s)' % a
    return base